In [1]:
big_fasta = "/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.faa"

In [2]:
from nb_tools import run_cd_hit, cluster_map
import os.path as op
import os
import numpy as np
from collections import defaultdict
import itertools
import pandas as pd

In [3]:
out_fasta = big_fasta.replace(".faa","_c70.faa")
out_fasta

'/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90_c70.faa'

In [7]:
[out_fasta, clstr] = run_cd_hit(big_fasta, out_fasta, c=0.7)

Running CD-HIT on /mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.faa


In [2]:
clstr = "/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90_c50.faa.clstr"

In [4]:
!head {clstr}

>Cluster 0
AG-908-B04_00191 hypothetical protein*
>Cluster 1
AG-913-A08_00038 hypothetical protein*
>Cluster 2
AG-892-O04_00030 hypothetical protein
AG-893-I07_00998 Pesticidal crystal protein Cry22Aa*
>Cluster 3
AG-913-A08_00040 hypothetical protein*
>Cluster 4


In [3]:
singles = True
cluster_map = defaultdict(list)
with open(clstr) as fh:
    for cluster_start, group in itertools.groupby(fh, lambda l: l[0] == '>'):
        members = []
        rep_seq = ''
        if not cluster_start: 
            for line in group:
                if "*" in line: 
                    rep_seq = line.strip().replace("*", "")
                else:
                    members.append(line.strip())
        if singles:
            cluster_map[rep_seq] = members
        elif len(members) > 0:
            cluster_map[rep_seq] = members
        else:
            continue

In [4]:
cm = cluster_map

In [5]:
len(cm)

469107

In [17]:
max_len = 0


for k in cm:
    if len(cm[k]) > max_len:
        max_len = len(cm[k])
        largest = k

In [18]:
largest

'AG-900-L05_00507 1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino] imidazole-4-carboxamide isomerase'

In [19]:
len(cm[largest])

236

In [21]:
count = 0
number = 0

for k in cm:
    seqs = [k] + cm[k]
    count += len(seqs)
    with open("./outputs/bats248_cdhit90_group{}.txt".format(number), "a") as oh:
        print("\n".join(seqs), file=oh)
    
    if count > 50000:
        count = 0
        number += 1
        print("resetting count", number)

resetting count 1
resetting count 2
resetting count 3
resetting count 4
resetting count 5
resetting count 6
resetting count 7
resetting count 8
resetting count 9
resetting count 10
resetting count 11
resetting count 12
resetting count 13
resetting count 14
resetting count 15
resetting count 16


In [18]:
lengths = {k:len(cm[k]) + 1 for k in cm}

In [19]:
df = pd.DataFrame.from_dict(lengths, orient='index').reset_index()

In [20]:
df.rename(columns={'index':'seed',0:'count'}, inplace=True)

In [21]:
singles = df[df['count'] == 1]

In [6]:
from pyfaidx import Fasta
from nb_tools import write_fa_record

In [7]:
fa = Fasta("/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.faa")

In [8]:
rec = fa['AG-915-A01_01267']

In [15]:
str(rec)

'MKSIFFYTVGKYLNYLLLFVRGILLAATFDYATYASWGIVMYVLSYFPIFGLGIPNIVLTNLKDFEVNSDECAKLAGSSILFIIYLCGIYLSIVFLLQITSFLQLDIINYYVLTLLVALYLIVDVLRNVARYSNRYLAIWTTEFFAIIPLLLLLVFKPVEITLNLSVSVITISTLLGMFILLKATRVIFKINDFVPFVRLIWKLGIPLLLYNYASYLLFLILRYFVLYSYDDTTVANFNFGWLIANGVILGLNIINWYFYPSLLKNLSAKNDPKFRISQKEIFAIQFLIAAVVLGLIPLIFEFLVTGYFTKYQYSITHFKYLLTSQLIFYLAYYPSTLLVVEERNRVLIKSGLMVSGIFAITIWVNNSIGGQIELFLLYSLLIVSTILFYFYLSYSTIWKGPRLPFFISIVSLLLLGLINNPWYKIVVFGVVLIIGVLWIDQIKSFMKKIKYELGSI'

In [27]:
missing = []

with open('./outputs/10_seqs.fasta', "w") as oh1, \
    open("./outputs/100_seqs.fasta","w") as oh2, \
    open("./outputs/1000_seqs.fasta","w") as oh3, \
    open("./outputs/10000_seqs.fasta","w") as oh4, \
    open("./outputs/100000_seqs.fasta","w") as oh5:
        for i, l in singles.iterrows():
        
            try:
                rec = fa[l['seed'].split()[0]]
            except:
                missing.append(l['seed'])
                continue
                
            if i < 13:
                write_fa_record(rec.long_name, str(rec), oh1)
            if i < 130:
                write_fa_record(rec.long_name, str(rec), oh2)
            if i < 1300:
                write_fa_record(rec.long_name, str(rec), oh3)
            if i < 13000:
                write_fa_record(rec.long_name, str(rec), oh4)
            if i < 130000:
                write_fa_record(rec.long_name, str(rec), oh5)
            else:
                break
            


In [24]:
singles

,seed,count
0,,1
1,AG-903-I02_00478 hypothetical protein,1
2,AG-919-L02_00037 hypothetical protein,1
4,AG-907-C13_00842 hypothetical protein,1
6,"AG-916-B09_01209 UDP-4-amino-4,6-dideoxy-N-ace...",1
7,AG-915-O18_00162 hypothetical protein,1
8,AG-915-L04_00200 Methionyl-tRNA formyltransferase,1
9,AG-918-O17_00099 hypothetical protein,1
10,AG-359-K08_00266 hypothetical protein,1
11,AG-895-F08_00105 hypothetical protein,1
